CC5213 - Recuperación de Información Multimedia
==

Slides 1.4 - Detección de Bordes
==

In [ ]:
import sys
import os
import numpy
import cv2
from PyQt5.QtWidgets import QApplication, QFileDialog

def mostrar_imagen(window_name, imagen):
    MAX_WIDTH = 700
    MAX_HEIGHT = 500
    if imagen.shape[0] > MAX_HEIGHT or imagen.shape[1] > MAX_WIDTH:
        #reducir tamaño
        fh = MAX_HEIGHT / imagen.shape[0]
        fw = MAX_WIDTH / imagen.shape[1]
        escala = min(fh, fw)
        imagen = cv2.resize(imagen, (0,0), fx=escala, fy=escala)
    #mostrar en pantalla
    cv2.imshow(window_name, imagen)

def ui_select_video():
    app = QApplication(list());
    options = QFileDialog.Options()
    filename, _ = QFileDialog.getOpenFileName(None, "Videos", ".", "Videos (*.mp4 *.mpg *.avi)", options=options)
    if not filename:
        filename = "0" # id de la webcam: 0=primera webcam, 1=segunda webcam
    return filename

def abrir_video(filename):
    if filename is None:
        filename = 0
    elif filename.isdigit():
        filename = int(filename)
    capture = None
    if isinstance(filename, int):
        print("abriendo webcam {}...".format(filename))
        capture = cv2.VideoCapture(filename)
    if (os.path.isfile(filename)):
        print("abriendo video {}...".format(filename))
        capture = cv2.VideoCapture(filename)
    if capture is None or not capture.isOpened():
        raise Exception("no puedo abrir video {}".format(filename))
    return capture;

def agregar_texto(imagen, texto):
    fontFace = cv2.FONT_HERSHEY_SIMPLEX
    fontScale = 0.7
    thickness = 2
    textSize = cv2.getTextSize(texto, fontFace, fontScale, thickness)
    position1 = (10,30)
    position2 = (20,40)
    cv2.rectangle(imagen, position1, position2, (0,255,0), -1)
    position3 = (10,30)
    cv2.putText(imagen, texto, position3, fontFace, fontScale, (200,0,0), thickness, cv2.LINE_AA)

def normalizar(imagen, valorAbsoluto = False, min0Max255 = False):
    img = imagen
    if valorAbsoluto:
        img = numpy.abs(imagen)
    if min0Max255:
        img2 = cv2.normalize(img, dst=None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
        img = img2
    return img

print("Usando OpenCV {} con Python {}.{}.{}".format(cv2.__version__, sys.version_info.major, sys.version_info.minor, sys.version_info.micro))


## Ejemplo 1 - Sobel

In [ ]:
def run_ejemplo(filename):
    global sobel_threshold, delta
    capture = abrir_video(filename)
    while capture.grab():
        retval, frame = capture.retrieve()
        if not retval:
            continue
        #convertir a gris
        frame_gris = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        mostrar_imagen("VIDEO", frame_gris)
        #calcular filtro de sobel
        sobelX = cv2.Sobel(frame_gris, ddepth=cv2.CV_32F, dx=1, dy=0, ksize=3)
        sobelY = cv2.Sobel(frame_gris, ddepth=cv2.CV_32F, dx=0, dy=1, ksize=3)
        mostrar_imagen("X", normalizar(sobelX, valorAbsoluto=True, min0Max255=True))
        mostrar_imagen("Y", normalizar(sobelY, valorAbsoluto=True, min0Max255=True))
        #magnitud del gradiente
        magnitud = numpy.sqrt(numpy.square(sobelX) + numpy.square(sobelY) )
        mostrar_imagen("MAGNITUD GRADIENTE", normalizar(magnitud, min0Max255=True))
        #aproximacion de la magnitud del gradiente
        #aprox = numpy.abs(sobelX) + numpy.abs(sobelY)
        #mostrar_imagen("APROX GRADIENTE", normalizar(aprox, min0Max255=True))
        #umbral sobre la magnitud del gradiente
        retval, bordes = cv2.threshold(magnitud, thresh=sobel_threshold, maxval=255, type=cv2.THRESH_BINARY)
        agregar_texto(bordes, "th={}".format(sobel_threshold))
        mostrar_imagen("BORDES", bordes)
        #esperar por una tecla
        key = cv2.waitKey(10)
        if key == ord(' '):
            key = cv2.waitKey(0)
        if key == ord('q') or key == 27:
            break
        elif key == ord('a'):
            sobel_threshold += delta
        elif key == ord('z'):
            if sobel_threshold - delta > 0:
                sobel_threshold -= delta
    capture.release()
    cv2.destroyAllWindows()

sobel_threshold = 51
delta = 5

filename = ui_select_video()
run_ejemplo(filename)

print("FIN")

## Ejemplo 2 - Canny

In [ ]:
def ejemplo(filename):
    global canny_threshold_1, canny_threshold_2, delta
    capture = abrir_video(filename)
    while capture.grab():
        retval, frame = capture.retrieve()
        if not retval:
            continue
        #convertir a gris
        frame_gris = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        mostrar_imagen("VIDEO", frame_gris)
        #calcular canny
        frame_canny = cv2.Canny(frame_gris, threshold1=canny_threshold_1, threshold2=canny_threshold_2)
        agregar_texto(frame_canny, "th={}-{}".format(canny_threshold_1, canny_threshold_2))
        mostrar_imagen("CANNY", frame_canny)
        #esperar por una tecla
        key = cv2.waitKey(10)
        if key == ord(' '):
            key = cv2.waitKey(0)
        if key == -1:
            continue
        elif key == ord('q') or key == 27:
            break
        elif key == ord('a'):
            canny_threshold_1 += delta
        elif key == ord('z'):
            if canny_threshold_1 - delta > 0:
                canny_threshold_1 -= delta
        elif key == ord('s'):
            canny_threshold_2 += delta
        elif key == ord('x'):
            if canny_threshold_2 - delta > 0:
                canny_threshold_2 -= delta
        else:
            print("unknown key '{}' ({})".format(chr(key), key))
    capture.release()
    cv2.destroyAllWindows()

canny_threshold_1 = 51
canny_threshold_2 = 301
delta = 10

filename = ui_select_video()
ejemplo(filename)

print("FIN")

## Ejemplo 3 - Difference of Gaussians (DoG)

In [ ]:
def ejemplo(filename):
    global sigma1, sigma2, threshold
    capture = abrir_video(filename)
    while capture.grab():
        retval, frame = capture.retrieve()
        if not retval:
            continue
        #convertir a gris
        frame_gris = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        mostrar_imagen("VIDEO", frame_gris)
        #calcular DoG
        blur1 = cv2.GaussianBlur(frame_gris, (sigma1, sigma1), 0)
        blur2 = cv2.GaussianBlur(frame_gris, (sigma2, sigma2), 0)
        frame_diff = cv2.subtract(blur1, blur2)
        mostrar_imagen("Diff", normalizar(frame_diff, min0Max255=True))
        threshold = round(threshold, 2)
        th, frame_bin = cv2.threshold(frame_diff, threshold, 255, cv2.THRESH_BINARY)
        agregar_texto(frame_bin, "th={}".format(threshold))
        mostrar_imagen("BIN", normalizar(frame_bin, min0Max255=True))
        #esperar por una tecla
        key = cv2.waitKey(10)
        if key == ord(' '):
            key = cv2.waitKey(0)
        if key == -1:
            continue
        elif key == ord('q') or key == 27:
            break
        elif key == ord('a'):
            threshold += delta
        elif key == ord('z'):
            if threshold - delta > 0:
                threshold -= delta
    capture.release()
    cv2.destroyAllWindows()

sigma1 = 3
sigma2 = 13
threshold = 5
delta = 0.05

filename = ui_select_video()
ejemplo(filename)

print("FIN")